<a href="https://colab.research.google.com/github/IqmanS/NLP-Assignments/blob/main/NLP_in_Python_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [ ]:
!wget https://github.com/IqmanS/NLP-Assignments/raw/main/data/dtm.pkl -q
!wget https://github.com/IqmanS/NLP-Assignments/raw/main/data/cv.pkl -q
!wget https://github.com/IqmanS/NLP-Assignments/raw/main/data/data_clean.pkl -q

In [ ]:
# Let's read in our document-term matrix
import pandas as pd
from IPython.display import clear_output
import pickle

data = pd.read_pickle('dtm.pkl')
data

,aaaaaah,aaaaah,aaah,aah,ab,abc,abcs,abigail,ability,abject,...,ziploced,zombie,zombies,zone,zones,zoning,zoo,zoom,zoomed,zuckerberg
ali,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
amy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beth,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,1,0,0,0,0,1,0,0,0,...,0,1,1,0,0,1,0,0,0,0
burr,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
dave,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dylan,0,0,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse

In [ ]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,amy,anthony,beth,bill,burr,dave,dylan,hasan,jim,...,kevin,lewis,louis,matt,mike,pete,ricky,sammy,trevor,wanda
aaaaaah,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
aaaaah,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaah,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aah,0,0,0,0,0,0,0,0,0,0,...,1,0,3,1,0,1,0,0,0,2
ab,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [ ]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.023*"like" + 0.015*"just" + 0.014*"im" + 0.013*"know" + 0.013*"dont" + 0.010*"right" + 0.008*"people" + 0.008*"thats" + 0.007*"gonna" + 0.007*"youre"'),
 (1,
  '0.032*"like" + 0.020*"know" + 0.016*"im" + 0.014*"just" + 0.014*"dont" + 0.009*"thats" + 0.008*"right" + 0.007*"youre" + 0.007*"people" + 0.006*"got"')]

In [ ]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.001*"like" + 0.000*"im" + 0.000*"just" + 0.000*"dont" + 0.000*"know" + 0.000*"people" + 0.000*"right" + 0.000*"thats" + 0.000*"time" + 0.000*"youre"'),
 (1,
  '0.019*"like" + 0.014*"im" + 0.013*"know" + 0.012*"dont" + 0.010*"said" + 0.009*"just" + 0.009*"got" + 0.007*"thats" + 0.007*"gonna" + 0.006*"shit"'),
 (2,
  '0.032*"like" + 0.019*"know" + 0.017*"just" + 0.016*"im" + 0.014*"dont" + 0.010*"right" + 0.009*"thats" + 0.008*"people" + 0.008*"youre" + 0.006*"yeah"')]

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.034*"like" + 0.015*"know" + 0.012*"just" + 0.011*"im" + 0.010*"dont" + 0.006*"goes" + 0.006*"youre" + 0.006*"thats" + 0.005*"mom" + 0.005*"think"'),
 (1,
  '0.034*"like" + 0.019*"know" + 0.017*"just" + 0.016*"im" + 0.014*"dont" + 0.009*"thats" + 0.009*"right" + 0.008*"people" + 0.007*"said" + 0.007*"gonna"'),
 (2,
  '0.024*"like" + 0.017*"im" + 0.016*"know" + 0.015*"dont" + 0.012*"just" + 0.009*"thats" + 0.008*"people" + 0.008*"youre" + 0.007*"right" + 0.007*"got"'),
 (3,
  '0.018*"like" + 0.017*"just" + 0.016*"know" + 0.014*"right" + 0.011*"im" + 0.011*"fcking" + 0.010*"dont" + 0.007*"thats" + 0.007*"people" + 0.007*"yeah"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [ ]:
# Let's create a function to pull out nouns from a string of text
import nltk
from nltk import word_tokenize, pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
ali,ali wong baby cobra transcript scraps from...
amy,amy schumer emergency contact transcript s...
anthony,anthony jeselnik thoughts and prayers full ...
beth,beth stelling girl daddy transcript scraps...
bill,bill burr im sorry you feel that way full t...
burr,bill burr paper tiger transcript scraps fr...
dave,dave chappelle the age of spin transcript ...
dylan,dylan moran off the hook transcript scraps...
hasan,hasan minhaj at white house correspondents d...
jim,jim jefferies bare full transcript scraps ...


In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ali baby cobra transcript scraps skip reviewsm...
amy,schumer emergency contact transcript scraps sk...
anthony,anthony jeselnik thoughts prayers transcript s...
beth,beth girl daddy transcript scraps skip reviews...
bill,bill sorry way transcript scraps skip reviewsm...
burr,bill paper tiger transcript scraps skip review...
dave,chappelle age spin transcript scraps skip revi...
dylan,dylan hook transcript scraps skip reviewsmovie...
hasan,minhaj house correspondents transcript scraps ...
jim,jim jefferies transcript scraps skip reviewsmo...


In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=list(stop_words))
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aaaaaah,aah,ab,abc,abcs,ability,abolitionists,abortingthats,abortion,abortionbut,...,zillion,zionists,zip,zombie,zombies,zone,zones,zoo,zoom,zuckerberg
ali,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
amy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
beth,0,0,0,0,0,0,0,1,4,1,...,0,0,1,0,0,0,0,0,0,0
bill,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
burr,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dylan,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"thing" + 0.009*"man" + 0.008*"hes" + 0.008*"life" + 0.008*"day" + 0.008*"cause" + 0.008*"gon" + 0.007*"shit" + 0.006*"way" + 0.006*"guy"'),
 (1,
  '0.009*"man" + 0.006*"cause" + 0.006*"day" + 0.006*"thing" + 0.005*"comedy" + 0.005*"lot" + 0.005*"shit" + 0.005*"fuck" + 0.005*"way" + 0.005*"life"')]

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.007*"joke" + 0.006*"comedy" + 0.006*"thing" + 0.005*"day" + 0.005*"hes" + 0.005*"years" + 0.004*"life" + 0.004*"way" + 0.004*"man" + 0.004*"gon"'),
 (1,
  '0.013*"man" + 0.009*"thing" + 0.009*"shit" + 0.009*"cause" + 0.008*"hes" + 0.008*"gon" + 0.008*"day" + 0.007*"life" + 0.007*"guy" + 0.006*"way"'),
 (2,
  '0.008*"thing" + 0.007*"cause" + 0.007*"life" + 0.007*"day" + 0.006*"lot" + 0.006*"way" + 0.006*"things" + 0.005*"man" + 0.005*"hes" + 0.005*"guy"')]

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.008*"thing" + 0.008*"day" + 0.006*"man" + 0.006*"hes" + 0.006*"way" + 0.006*"gon" + 0.006*"things" + 0.006*"id" + 0.006*"cause" + 0.005*"life"'),
 (1,
  '0.008*"day" + 0.008*"cause" + 0.007*"thing" + 0.007*"mom" + 0.007*"man" + 0.007*"lot" + 0.007*"hes" + 0.006*"life" + 0.006*"way" + 0.005*"guy"'),
 (2,
  '0.012*"man" + 0.012*"thing" + 0.011*"gon" + 0.011*"shit" + 0.010*"life" + 0.010*"hes" + 0.008*"guy" + 0.007*"cause" + 0.007*"day" + 0.006*"way"'),
 (3,
  '0.011*"man" + 0.009*"shit" + 0.007*"cause" + 0.007*"thing" + 0.006*"lot" + 0.006*"life" + 0.006*"kids" + 0.006*"things" + 0.005*"way" + 0.005*"theyre"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [ ]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return ' '.join(nouns_adj)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ali wong baby cobra transcript scraps loft ski...
amy,amy schumer emergency contact transcript scrap...
anthony,anthony jeselnik thoughts prayers full transcr...
beth,beth girl daddy transcript scraps loft skip mo...
bill,bill im sorry way full transcript scraps loft ...
burr,bill paper tiger transcript scraps loft skip m...
dave,chappelle age spin transcript scraps loft skip...
dylan,dylan hook transcript scraps loft skip moviesm...
hasan,hasan minhaj white house correspondents full t...
jim,jim jefferies full transcript scraps loft skip...


In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=list(stop_words), max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaaah,aaaaah,aah,ab,abc,abcs,ability,abject,able,ablebodied,...,zillion,zionists,zip,zombie,zombies,zone,zones,zoo,zoom,zuckerberg
ali,0,0,0,0,1,0,0,0,2,0,...,0,0,0,1,0,0,0,0,0,0
amy,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beth,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
bill,0,1,0,0,0,1,0,0,1,0,...,1,0,0,1,1,0,0,0,0,0
burr,0,0,0,0,0,0,0,0,6,1,...,0,0,0,0,0,0,0,1,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dylan,0,0,0,1,0,0,0,0,4,0,...,0,0,0,1,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"fuck" + 0.004*"mom" + 0.004*"joke" + 0.004*"kid" + 0.004*"fcking" + 0.003*"baby" + 0.003*"fck" + 0.003*"ass" + 0.003*"dude" + 0.003*"girl"'),
 (1,
  '0.006*"fuck" + 0.004*"fucking" + 0.004*"dude" + 0.002*"son" + 0.002*"somebody" + 0.002*"husband" + 0.002*"ahah" + 0.002*"anthem" + 0.002*"na" + 0.002*"days"')]

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"fcking" + 0.006*"fucking" + 0.006*"fuck" + 0.006*"fck" + 0.005*"dude" + 0.004*"kid" + 0.003*"ass" + 0.003*"joke" + 0.003*"sht" + 0.003*"baby"'),
 (1,
  '0.005*"fuck" + 0.005*"mom" + 0.003*"phone" + 0.003*"ass" + 0.003*"baby" + 0.003*"bitch" + 0.002*"yall" + 0.002*"kid" + 0.002*"dude" + 0.002*"girl"'),
 (2,
  '0.005*"course" + 0.004*"somebody" + 0.004*"joke" + 0.004*"dog" + 0.003*"fuck" + 0.003*"mom" + 0.003*"anthony" + 0.003*"young" + 0.003*"men" + 0.002*"tit"')]

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.008*"fcking" + 0.005*"somebody" + 0.005*"course" + 0.005*"sht" + 0.005*"dog" + 0.004*"bitch" + 0.004*"ass" + 0.004*"fck" + 0.004*"wife" + 0.003*"phone"'),
 (1,
  '0.006*"joke" + 0.005*"mom" + 0.003*"parents" + 0.003*"anthem" + 0.003*"kid" + 0.003*"baby" + 0.003*"na" + 0.003*"america" + 0.003*"anthony" + 0.002*"clinton"'),
 (2,
  '0.005*"mom" + 0.005*"fck" + 0.004*"dude" + 0.004*"fuck" + 0.004*"girl" + 0.003*"kid" + 0.003*"jenny" + 0.003*"fcking" + 0.003*"president" + 0.003*"phone"'),
 (3,
  '0.009*"fuck" + 0.007*"fucking" + 0.005*"dude" + 0.003*"son" + 0.003*"pussy" + 0.003*"men" + 0.003*"ass" + 0.003*"kid" + 0.003*"joke" + 0.003*"somebody"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.010*"fcking" + 0.008*"fck" + 0.005*"dude" + 0.004*"country" + 0.004*"sht" + 0.004*"pussy" + 0.004*"anthem" + 0.004*"america" + 0.003*"president" + 0.003*"joke"'),
 (1,
  '0.006*"joke" + 0.004*"baby" + 0.003*"fuck" + 0.003*"kid" + 0.003*"dude" + 0.003*"jenny" + 0.003*"anthony" + 0.003*"jokes" + 0.003*"dead" + 0.003*"na"'),
 (2,
  '0.011*"fuck" + 0.009*"fucking" + 0.007*"mom" + 0.005*"dude" + 0.004*"kid" + 0.004*"son" + 0.003*"gun" + 0.002*"anybody" + 0.002*"cats" + 0.002*"dick"'),
 (3,
  '0.006*"fuck" + 0.004*"somebody" + 0.004*"ass" + 0.004*"mom" + 0.004*"men" + 0.003*"son" + 0.003*"wife" + 0.003*"baby" + 0.003*"fucking" + 0.003*"phone"')]

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
sorted(list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index)))

[(0, 'burr'),
 (0, 'hasan'),
 (0, 'joe'),
 (0, 'louis'),
 (0, 'sammy'),
 (0, 'trevor'),
 (1, 'ali'),
 (1, 'anthony'),
 (1, 'matt'),
 (1, 'mike'),
 (1, 'ricky'),
 (2, 'bill'),
 (2, 'lewis'),
 (2, 'pete'),
 (3, 'amy'),
 (3, 'beth'),
 (3, 'dave'),
 (3, 'dylan'),
 (3, 'jim'),
 (3, 'john'),
 (3, 'kathleen'),
 (3, 'kevin'),
 (3, 'wanda')]

* Topic 0: [hasan, joe, louis, mike]
* Topic 1: [ali, amy, anthony, beth, dave, dylan, john, lewis]
* Topic 2: [jim, matt, trevor]
* Topic 3: [bill, burr, kathleen, kevin, pete, ricky, sammy, wanda]

### Assignment 1:
Try further modifying the parameters of the topic models above and see if you can get better topics.

In [ ]:
# chunksize: controls how many documents are processed at a time in the training algorithm
# passes: controls how often we train the model on the entire corpus (like epochs)
# alpha = 'auto' and eta = 'auto'. Again this is somewhat technical, but essentially we are automatically learning two parameters in the model that we usually would have to specify explicitly.



ldana = models.LdaModel(corpus=corpusna, id2word=id2wordna,
                        num_topics=3,
                        chunksize=10,
                        passes=2,
                        alpha='auto')
ldana.print_topics()

[(0,
  '0.005*"joke" + 0.004*"fuck" + 0.004*"baby" + 0.003*"mom" + 0.003*"kid" + 0.003*"mad" + 0.002*"wife" + 0.002*"fucking" + 0.002*"phone" + 0.002*"bitch"'),
 (1,
  '0.006*"fuck" + 0.004*"mom" + 0.003*"somebody" + 0.003*"fucking" + 0.003*"dude" + 0.002*"course" + 0.002*"na" + 0.002*"phone" + 0.002*"ok" + 0.002*"young"'),
 (2,
  '0.007*"anthem" + 0.007*"pussy" + 0.007*"country" + 0.006*"america" + 0.005*"berkeley" + 0.005*"um" + 0.005*"germany" + 0.005*"number" + 0.005*"song" + 0.005*"french"')]

In [ ]:
ldana = models.LdaModel(corpus=corpusna, id2word=id2wordna,
                        num_topics=3,
                        chunksize=1,
                        passes=100,
                        alpha='auto')

clear_output(wait=True)
ldana.print_topics()

[(0,
  '0.007*"fuck" + 0.005*"mom" + 0.005*"joke" + 0.004*"fucking" + 0.004*"dude" + 0.004*"kid" + 0.004*"ass" + 0.003*"baby" + 0.003*"phone" + 0.003*"somebody"'),
 (1,
  '0.008*"anthem" + 0.007*"fck" + 0.007*"fcking" + 0.006*"germany" + 0.003*"museums" + 0.003*"chimp" + 0.003*"hampstead" + 0.003*"twitter" + 0.003*"stalker" + 0.003*"tweet"'),
 (2,
  '0.007*"jenny" + 0.003*"clinton" + 0.003*"relationship" + 0.003*"scrambler" + 0.003*"john" + 0.003*"apartment" + 0.003*"sort" + 0.003*"cow" + 0.003*"andy" + 0.003*"ok"')]

### Assignment 2:
Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

In [ ]:
'''Given a string of text, tokenize the text and use only the PRONOUNS.''' # Personal pronoun, Possessive pronoun, Wh-pronoun
def pronouns(text):
    is_pronoun = lambda pos: pos[:3] == 'PRP' or pos[:2] == "WP"
    tokenized = word_tokenize(text)
    all_pronouns = [word for (word, pos) in pos_tag(tokenized) if is_pronoun(pos)]
    return ' '.join(all_pronouns)

In [ ]:
# Apply the pronouns function to the transcripts to filter only on pronouns & Create a new document-term matrix using only pronouns
data_pronouns = pd.DataFrame(data_clean.transcript.apply(pronouns))

cv_pronouns = CountVectorizer(stop_words=list(stop_words))
data_cv_pronouns = cv_pronouns.fit_transform(data_pronouns.transcript)
data_dtm_pronouns = pd.DataFrame(data_cv_pronouns.toarray(), columns=cv_pronouns.get_feature_names_out())
data_dtm_pronouns.index = data_pronouns.index

In [ ]:
# Create the corpus for pronouns
corpus_pronouns = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtm_pronouns.transpose()))

# Create the vocabulary dictionary for pronouns
id2word_pronouns = dict((v, k) for k, v in cv_pronouns.vocabulary_.items())

In [ ]:
# Attempt LDA with pronouns
lda_pronouns = models.LdaModel(corpus=corpus_pronouns, id2word=id2word_pronouns,
                               num_topics=5)
lda_pronouns.print_topics()

[(0,
  '0.183*"oh" + 0.164*"ya" + 0.056*"youve" + 0.056*"whichever" + 0.056*"youll" + 0.056*"thatwhat" + 0.044*"yes" + 0.044*"hey" + 0.043*"luck" + 0.040*"weed"'),
 (1,
  '0.130*"okay" + 0.072*"youve" + 0.071*"youd" + 0.071*"ya" + 0.071*"huh" + 0.071*"nightmarethat" + 0.071*"brother" + 0.071*"oj" + 0.070*"lotthat" + 0.013*"oh"'),
 (2,
  '0.116*"oh" + 0.089*"ya" + 0.056*"wan" + 0.056*"wetthat" + 0.056*"yummy" + 0.056*"pregnantthat" + 0.056*"youlast" + 0.056*"mei" + 0.056*"em" + 0.056*"wont"'),
 (3,
  '0.121*"warmth" + 0.121*"ok" + 0.119*"shat" + 0.022*"oh" + 0.021*"wan" + 0.021*"youve" + 0.021*"youd" + 0.021*"hey" + 0.021*"okay" + 0.021*"lotthat"'),
 (4,
  '0.116*"hang" + 0.116*"wan" + 0.065*"oh" + 0.064*"okay" + 0.064*"roganwhat" + 0.064*"hat" + 0.064*"rightthat" + 0.064*"jenny" + 0.063*"youve" + 0.062*"hey"')]